In [1]:
import datetime as dt
from datetime import date
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn import preprocessing

# Data Input, EDA & Preprocessing

## Individual Data Input

In [2]:
price_vol = pd.read_csv('../data/Price & Volume BTC.csv')
tweets = pd.read_csv('../data/Tweets.csv')
funding_rates_1600 = pd.read_csv('../data/Funding Rates BTCUSDT 1600.csv')
funding_rates_0800 = pd.read_csv('../data/Funding Rates BTCUSDT 0800.csv')
funding_rates_0000 = pd.read_csv('../data/Funding Rates BTCUSDT 0000.csv')
google = pd.read_csv('../data/Google.csv')
transactions = pd.read_csv('../data/Transactions.csv')
unique_addresses = pd.read_csv('../data/Unique Addresses.csv')
active_addresses = pd.read_csv('../data/Active Addresses.csv')

## Data Compilation

In [3]:
# get daily ave value for twitter attributes
tweets = tweets.groupby(by="Time").mean()
tweets = tweets.reset_index()

df = pd.merge(price_vol, funding_rates_0000, on=['Time'])
df = pd.merge(df, funding_rates_0800, on=['Time'])
df = pd.merge(df, funding_rates_1600, on=['Time'])
df = pd.merge(df, google, on=['Time'])
df = pd.merge(df, active_addresses, on=['Time'])
df = pd.merge(df, unique_addresses, on=['Time'])
df = pd.merge(df, transactions, on=['Time'])
df = pd.merge(df, tweets, how="outer")
df.head()

,Time,Open,Close,High,Low,Volume,Funding Rate 0000,Funding Rate 0800,Funding Rate 1600,Bitcoin_SVI,Cryptocurrency_SVI,No. of Active Addresses,No. of Unique Addresses,Transactions,Favorites,Retweets,Compound,Negative,Positive,Neutral
0,2022-03-03,43896.8,42456.3,44088.0,41832.0,324545814.8,0.01%,0.01%,0.01%,13.84,24.08,1011448,701198.0,268244.0,443.269231,79.576923,0.109327,0.036269,0.063731,0.900000
1,2022-03-02,44428.2,43896.9,45367.5,43332.8,340460526.5,0.00%,0.00%,0.01%,15.61,26.33,1091796,731603.0,288657.0,534.200000,90.366667,0.068437,0.037100,0.068967,0.893967
2,2022-03-01,43174.8,44428.2,44966.0,42796.9,444858241.2,-0.01%,0.01%,0.01%,17.67,29.34,1065354,722863.0,280634.0,353.514286,61.028571,0.006329,0.062714,0.070200,0.867114
3,2022-02-28,37706.7,43179.8,44210.4,37446.5,564121052.5,0.00%,-0.02%,-0.01%,16.20,24.83,981066,702483.0,270918.0,332.333333,61.861111,0.135383,0.033806,0.073306,0.892861
4,2022-02-27,39117.3,37706.7,39864.0,36977.4,388368905.7,0.01%,0.01%,0.00%,12.96,21.07,768705,583361.0,211164.0,128.000000,30.000000,0.106867,0.022667,0.055000,0.922333


In [4]:
print("Type of variables: ", "\n", df.dtypes)

Type of variables:  
 Time                        object
Open                       float64
Close                      float64
High                       float64
Low                        float64
Volume                     float64
Funding Rate 0000           object
Funding Rate 0800           object
Funding Rate 1600           object
Bitcoin_SVI                float64
Cryptocurrency_SVI         float64
No. of Active Addresses      int64
No. of Unique Addresses    float64
Transactions               float64
Favorites                  float64
Retweets                   float64
Compound                   float64
Negative                   float64
Positive                   float64
Neutral                    float64
dtype: object


# EDA & Preprocessing

In [5]:
# Select features, the target output is'Close'
feature_pool = ['Time', 'Favorites','Retweets','Compound','Negative','Positive',\
                'Neutral','Open', 'High','Low','Close',\
                'Volume','Funding Rate 0000','Funding Rate 0800','Funding Rate 1600','Bitcoin_SVI',\
                'Cryptocurrency_SVI','No. of Active Addresses','No. of Unique Addresses','Transactions']

#Note : Close is a feature as well 

output = 'Close'

## Data Type Transformation

In [6]:
# Transform object type of "Funding Rate 0000", "Funding Rate 0800" and " Funding Rate 1600" to numerical type
df['Funding Rate 0000'] = df['Funding Rate 0000'].map(lambda x: float('nan') if pd.isnull(x) else float(x.replace('%','')))
df['Funding Rate 0800'] = df['Funding Rate 0800'].map(lambda x: float('nan') if pd.isnull(x) else float(x.replace('%','')))
df['Funding Rate 1600'] = df['Funding Rate 1600'].map(lambda x: float('nan') if pd.isnull(x) else float(x.replace('%','')))

df['Month'] = df['Time'].str[:7]
df['Time'] = df['Time'].map(lambda x: int('nan') if pd.isnull(x) else int(x.replace("-", "")))
df['Month'] = df['Month'].map(lambda x: int('nan') if pd.isnull(x) else int(x.replace("-", "")))

df.head(10)


,Time,Open,Close,High,Low,Volume,Funding Rate 0000,Funding Rate 0800,Funding Rate 1600,Bitcoin_SVI,...,No. of Active Addresses,No. of Unique Addresses,Transactions,Favorites,Retweets,Compound,Negative,Positive,Neutral,Month
0,20220303,43896.8,42456.3,44088.0,41832.0,324545814.8,0.01,0.01,0.01,13.84,...,1011448,701198.0,268244.0,443.269231,79.576923,0.109327,0.036269,0.063731,0.900000,202203
1,20220302,44428.2,43896.9,45367.5,43332.8,340460526.5,0.00,0.00,0.01,15.61,...,1091796,731603.0,288657.0,534.200000,90.366667,0.068437,0.037100,0.068967,0.893967,202203
2,20220301,43174.8,44428.2,44966.0,42796.9,444858241.2,-0.01,0.01,0.01,17.67,...,1065354,722863.0,280634.0,353.514286,61.028571,0.006329,0.062714,0.070200,0.867114,202203
3,20220228,37706.7,43179.8,44210.4,37446.5,564121052.5,0.00,-0.02,-0.01,16.20,...,981066,702483.0,270918.0,332.333333,61.861111,0.135383,0.033806,0.073306,0.892861,202202
4,20220227,39117.3,37706.7,39864.0,36977.4,388368905.7,0.01,0.01,0.00,12.96,...,768705,583361.0,211164.0,128.000000,30.000000,0.106867,0.022667,0.055000,0.922333,202202
5,20220226,39219.6,39119.1,40300.0,38560.3,306410979.9,0.00,0.01,0.01,14.37,...,818914,582374.0,220423.0,863.800000,178.600000,0.131040,0.070000,0.085000,0.845000,202202
6,20220225,38320.9,39219.6,39695.0,38015.0,386047125.7,-0.01,0.01,0.01,18.61,...,936543,687447.0,260340.0,453.100000,83.850000,0.136800,0.041950,0.079400,0.878650,202202
7,20220224,37248.2,38320.9,39823.3,34325.0,680985023.6,0.00,0.01,-0.02,27.43,...,958866,676575.0,254139.0,69.958333,22.375000,-0.021021,0.074313,0.069271,0.856375,202202
8,20220223,38229.6,37248.1,39285.2,37046.8,331098529.4,0.00,0.00,-0.01,14.37,...,859810,622682.0,251275.0,59.800000,19.485714,0.033834,0.051171,0.053629,0.895200,202202
9,20220222,37020.5,38229.6,38448.4,36333.1,324319840.5,0.00,-0.01,0.00,16.98,...,886571,647191.0,260265.0,193.058824,36.411765,0.185906,0.045647,0.086588,0.867706,202202


## Train-Test Split

In [7]:
from sklearn.model_selection import TimeSeriesSplit
X, y = df, df[['Time', output]]
tscv = TimeSeriesSplit()
for train_index, val_index in tscv.split(X):
  print("TRAIN:", train_index, "TEST:", val_index)
  X_train, X_test = X.iloc[train_index], X.iloc[val_index]
  y_train, y_test = y.iloc[train_index], y.iloc[val_index]

print(y_train.sum()/y_train.count(),y_test.sum()/y_test.count())


TRAIN: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124] TEST: [125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142
 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160
 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178
 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196
 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214
 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232
 233 234 235 236 237 238 239 240 241 242 

## Handle Missing Data

In [8]:
# Count missing value
print(df.isna().sum())

# check the consistency of columns with missing value between train set and the whole set
print(df[feature_pool].isnull().any().tolist()==X_train.isnull().any().tolist())
print(df[output].isnull().any().tolist()==y_train.isnull().any().tolist())

Time                         0
Open                         0
Close                        0
High                         0
Low                          0
Volume                       0
Funding Rate 0000            0
Funding Rate 0800            0
Funding Rate 1600            0
Bitcoin_SVI                  0
Cryptocurrency_SVI           0
No. of Active Addresses      0
No. of Unique Addresses      2
Transactions                 0
Favorites                  458
Retweets                   458
Compound                   458
Negative                   458
Positive                   458
Neutral                    458
Month                        0
dtype: int64
False
False


### Monthly Sentiment Analysis

In [9]:
tweets.insert(1, "Month", "NaN")

tweets['Month'] = tweets['Time'].str[:7]
tweets['Month'] = tweets['Month'].map(lambda x: int('nan') if pd.isnull(x) else int(x.replace("-", "")))
tweets.head(10)

,Time,Month,Favorites,Retweets,Compound,Negative,Positive,Neutral
0,2020-03-12,202003,5.0,2.0,0.0772,0.091,0.100,0.809
1,2020-03-17,202003,98.0,96.0,0.4019,0.000,0.114,0.886
2,2020-03-20,202003,125.0,95.0,0.4767,0.000,0.146,0.854
3,2020-03-30,202003,3.0,0.0,0.0803,0.061,0.071,0.869
4,2020-03-31,202003,186.0,142.0,0.6597,0.000,0.278,0.722
5,2020-04-01,202004,171.0,120.0,0.5267,0.061,0.184,0.754
6,2020-04-16,202004,419.0,94.0,0.0000,0.000,0.000,1.000
7,2020-04-22,202004,73.0,18.0,0.3182,0.000,0.084,0.916
8,2020-04-30,202004,994.0,153.0,0.0000,0.000,0.000,1.000
9,2020-05-09,202005,35.0,2.0,0.0000,0.000,0.000,1.000


In [10]:
tweets = tweets.groupby(by="Month").mean()
tweets = tweets.reset_index()

In [11]:
tweets.head(10)

,Month,Favorites,Retweets,Compound,Negative,Positive,Neutral
0,202003,83.400000,67.000000,0.339160,0.030400,0.141800,0.828000
1,202004,414.250000,96.250000,0.211225,0.015250,0.067000,0.917500
2,202005,283.285714,49.142857,0.002414,0.027857,0.057429,0.914571
3,202006,232.000000,155.000000,0.401900,0.000000,0.072000,0.928000
4,202007,303.750000,79.000000,0.227450,0.000000,0.065250,0.934750
5,202008,1103.925926,241.740741,0.379407,0.016296,0.146444,0.837407
6,202009,63.500000,17.000000,0.121117,0.000000,0.035000,0.965000
7,202010,300.000000,50.000000,0.405267,0.000000,0.146000,0.854000
8,202011,254.666667,37.833333,0.117625,0.049250,0.123000,0.827750
9,202012,487.428571,56.571429,0.133693,0.058571,0.110143,0.831286


In [12]:
df_X_train = pd.DataFrame(columns = ['Time', 'Month', 'Favorites', 'Retweets', 'Compound', \
                                 'Negative', 'Positive', 'Neutral', 'Open', 'High', \
                                 'Low', 'Close', 'Volume', 'Funding Rate 0000', 'Funding Rate 0800', \
                                 'Funding Rate 1600', 'Bitcoin_SVI', 'Cryptocurrency_SVI',  'No. of Active Addresses', \
                                 'No. of Unique Addresses', 'Transactions'])

for i, row in X_train.iterrows():
    if (pd.isna(X_train.loc[i].at["Favorites"])):
        data = tweets[tweets['Month'] == X_train.loc[i].at["Month"]]
        new_row = {'Time': X_train.loc[i].at["Time"], 'Month': X_train.loc[i].at["Month"], 'Favorites': data.Favorites.values[0], 'Retweets': data.Retweets.values[0], 'Compound': data.Compound.values[0], \
                   'Negative': data.Negative.values[0], 'Positive': data.Positive.values[0], 'Neutral': data.Neutral.values[0], 'Open': X_train.loc[i].at["Open"], \
                   'High': X_train.loc[i].at["High"], 'Low': X_train.loc[i].at["Low"], 'Close': X_train.loc[i].at["Close"], 'Volume': X_train.loc[i].at["Volume"], 'Funding Rate 0000': X_train.loc[i].at["Funding Rate 0000"], \
                   'Funding Rate 0800': X_train.loc[i].at["Funding Rate 0800"], 'Funding Rate 1600': X_train.loc[i].at["Funding Rate 1600"], 'Bitcoin_SVI': X_train.loc[i].at["Bitcoin_SVI"], \
                   'Cryptocurrency_SVI': X_train.loc[i].at["Cryptocurrency_SVI"], 'No. of Active Addresses': X_train.loc[i].at["No. of Active Addresses"], 'No. of Unique Addresses': X_train.loc[i].at["No. of Unique Addresses"], \
                   'Transactions': X_train.loc[i].at["Transactions"]}
        df_X_train = df_X_train.append(new_row, ignore_index=True)
    else:
        new_row = {'Time': X_train.loc[i].at["Time"], 'Month': X_train.loc[i].at["Month"], 'Favorites': X_train.loc[i].at["Favorites"], 'Retweets': X_train.loc[i].at["Retweets"], 'Compound': X_train.loc[i].at["Compound"], \
                   'Negative': X_train.loc[i].at["Negative"], 'Positive': X_train.loc[i].at["Positive"], 'Neutral': X_train.loc[i].at["Neutral"], 'Open': X_train.loc[i].at["Open"],\
                   'High': X_train.loc[i].at["High"], 'Low': X_train.loc[i].at["Low"],  'Close': X_train.loc[i].at["Close"],'Volume': X_train.loc[i].at["Volume"], 'Funding Rate 0000': X_train.loc[i].at["Funding Rate 0000"], \
                   'Funding Rate 0800': X_train.loc[i].at["Funding Rate 0800"], 'Funding Rate 1600': X_train.loc[i].at["Funding Rate 1600"], 'Bitcoin_SVI': X_train.loc[i].at["Bitcoin_SVI"], \
                   'Cryptocurrency_SVI': X_train.loc[i].at["Cryptocurrency_SVI"], 'No. of Active Addresses': X_train.loc[i].at["No. of Active Addresses"], 'No. of Unique Addresses': X_train.loc[i].at["No. of Unique Addresses"], \
                   'Transactions': X_train.loc[i].at["Transactions"]}
        df_X_train = df_X_train.append(new_row, ignore_index=True)


/var/folders/t1/pxny3xn56cd89t4tpk4s00vc0000gn/T/ipykernel_1904/2033116201.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_X_train = df_X_train.append(new_row, ignore_index=True)
/var/folders/t1/pxny3xn56cd89t4tpk4s00vc0000gn/T/ipykernel_1904/2033116201.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_X_train = df_X_train.append(new_row, ignore_index=True)
/var/folders/t1/pxny3xn56cd89t4tpk4s00vc0000gn/T/ipykernel_1904/2033116201.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_X_train = df_X_train.append(new_row, ignore_index=True)
/var/folders/t1/pxny3xn56cd89t4tpk4s00vc0000gn/T/ipykernel_1904/2033116201.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a 

In [13]:
df_X_test = pd.DataFrame(columns = ['Time', 'Month', 'Favorites', 'Retweets', 'Compound', \
                                 'Negative', 'Positive', 'Neutral', 'Open', 'High', \
                                 'Low', 'Close','Volume', 'Funding Rate 0000', 'Funding Rate 0800', \
                                 'Funding Rate 1600', 'Bitcoin_SVI', 'Cryptocurrency_SVI',  'No. of Active Addresses', \
                                 'No. of Unique Addresses', 'Transactions'])

for i, row in X_test.iterrows():
    if (pd.isna(X_test.loc[i].at["Favorites"])):
        data = tweets[tweets['Month'] == X_test.loc[i].at["Month"]]
        new_row = {'Time': X_test.loc[i].at["Time"], 'Month': X_test.loc[i].at["Month"], 'Favorites': data.Favorites.values[0], 'Retweets': data.Retweets.values[0], 'Compound': data.Compound.values[0], \
                   'Negative': data.Negative.values[0], 'Positive': data.Positive.values[0], 'Neutral': data.Neutral.values[0], 'Open': X_test.loc[i].at["Open"], \
                   'High': X_test.loc[i].at["High"], 'Low': X_test.loc[i].at["Low"],  'Close': X_test.loc[i].at["Close"],'Volume': X_test.loc[i].at["Volume"], 'Funding Rate 0000': X_test.loc[i].at["Funding Rate 0000"], \
                   'Funding Rate 0800': X_test.loc[i].at["Funding Rate 0800"], 'Funding Rate 1600': X_test.loc[i].at["Funding Rate 1600"], 'Bitcoin_SVI': X_test.loc[i].at["Bitcoin_SVI"], \
                   'Cryptocurrency_SVI': X_test.loc[i].at["Cryptocurrency_SVI"], 'No. of Active Addresses': X_test.loc[i].at["No. of Active Addresses"], 'No. of Unique Addresses': X_test.loc[i].at["No. of Unique Addresses"], \
                   'Transactions': X_test.loc[i].at["Transactions"]}
        df_X_test = df_X_test.append(new_row, ignore_index=True)
    else:
        new_row = {'Time': X_test.loc[i].at["Time"], 'Month': X_test.loc[i].at["Month"], 'Favorites': X_test.loc[i].at["Favorites"], 'Retweets': X_test.loc[i].at["Retweets"], 'Compound': X_test.loc[i].at["Compound"], \
                   'Negative': X_test.loc[i].at["Negative"], 'Positive': X_test.loc[i].at["Positive"], 'Neutral': X_test.loc[i].at["Neutral"], 'Open': X_test.loc[i].at["Open"], \
                   'High': X_test.loc[i].at["High"], 'Low': X_test.loc[i].at["Low"],  'Close': X_test.loc[i].at["Close"],'Volume': X_test.loc[i].at["Volume"], 'Funding Rate 0000': X_test.loc[i].at["Funding Rate 0000"], \
                   'Funding Rate 0800': X_test.loc[i].at["Funding Rate 0800"], 'Funding Rate 1600': X_test.loc[i].at["Funding Rate 1600"], 'Bitcoin_SVI': X_test.loc[i].at["Bitcoin_SVI"], \
                   'Cryptocurrency_SVI': X_test.loc[i].at["Cryptocurrency_SVI"], 'No. of Active Addresses': X_test.loc[i].at["No. of Active Addresses"], 'No. of Unique Addresses': X_test.loc[i].at["No. of Unique Addresses"], \
                   'Transactions': X_test.loc[i].at["Transactions"]}
        df_X_test = df_X_test.append(new_row, ignore_index=True)


/var/folders/t1/pxny3xn56cd89t4tpk4s00vc0000gn/T/ipykernel_1904/2155002156.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_X_test = df_X_test.append(new_row, ignore_index=True)
/var/folders/t1/pxny3xn56cd89t4tpk4s00vc0000gn/T/ipykernel_1904/2155002156.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_X_test = df_X_test.append(new_row, ignore_index=True)
/var/folders/t1/pxny3xn56cd89t4tpk4s00vc0000gn/T/ipykernel_1904/2155002156.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_X_test = df_X_test.append(new_row, ignore_index=True)
/var/folders/t1/pxny3xn56cd89t4tpk4s00vc0000gn/T/ipykernel_1904/2155002156.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

In [14]:
X_test = df_X_test
X_test = X_test.drop(['Month'], axis=1)
X_test

,Time,Favorites,Retweets,Compound,Negative,Positive,Neutral,Open,High,Low,Close,Volume,Funding Rate 0000,Funding Rate 0800,Funding Rate 1600,Bitcoin_SVI,Cryptocurrency_SVI,No. of Active Addresses,No. of Unique Addresses,Transactions
0,20200702.0,303.75,79.0,0.22745,0.0,0.06525,0.93475,9231.4,9260.4,8944.1,9086.1,22009723.03,0.01,0.01,0.01,4.51,3.64,951042.0,674745.0,339469.0
1,20200701.0,303.75,79.0,0.22745,0.0,0.06525,0.93475,9136.6,9289.0,9091.0,9231.4,19313717.22,0.01,0.01,0.01,4.33,3.47,1078007.0,758469.0,382570.0
2,20200630.0,232.0,155.0,0.4019,0.0,0.072,0.928,9191.1,9202.0,9067.9,9136.6,13205668.17,0.01,0.01,0.01,4.68,3.47,926238.0,621031.0,299733.0
3,20200629.0,232.0,155.0,0.4019,0.0,0.072,0.928,9119.1,9234.9,9027.4,9191.1,18164728.28,0.0,0.01,0.01,4.68,3.47,962232.0,625207.0,315424.0
4,20200628.0,232.0,155.0,0.4019,0.0,0.072,0.928,9012.8,9189.4,8933.2,9119.1,16167462.11,0.01,0.01,0.01,4.68,3.72,686270.0,500152.0,264909.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,20200308.0,83.4,67.0,0.33916,0.0304,0.1418,0.828,8889.9,8889.9,8001.1,8036.5,14397155.01,0.03,0.05,0.05,8.02,4.34,654911.0,501571.0,302694.0
117,20200307.0,83.4,67.0,0.33916,0.0304,0.1418,0.828,9133.9,9184.9,8789.8,8889.9,7525135.261,0.04,0.04,0.03,7.1,3.04,682432.0,526880.0,313312.0
118,20200306.0,83.4,67.0,0.33916,0.0304,0.1418,0.828,9056.2,9168.7,8989.1,9134.3,8833554.543,0.02,0.02,0.02,8.95,3.33,804945.0,601698.0,338383.0
119,20200305.0,83.4,67.0,0.33916,0.0304,0.1418,0.828,8750.8,9153.1,8750.7,9056.9,15218448.72,0.02,0.01,0.02,9.26,3.76,793221.0,603996.0,349529.0


In [15]:
X_train = df_X_train
X_train = X_train.drop(['Month'], axis=1)
X_train

,Time,Favorites,Retweets,Compound,Negative,Positive,Neutral,Open,High,Low,Close,Volume,Funding Rate 0000,Funding Rate 0800,Funding Rate 1600,Bitcoin_SVI,Cryptocurrency_SVI,No. of Active Addresses,No. of Unique Addresses,Transactions
0,20220303.0,443.269231,79.576923,0.109327,0.036269,0.063731,0.9,43896.8,44088.0,41832.0,42456.3,324545814.8,0.01,0.01,0.01,13.84,24.08,1011448.0,701198.0,268244.0
1,20220302.0,534.2,90.366667,0.068437,0.0371,0.068967,0.893967,44428.2,45367.5,43332.8,43896.9,340460526.5,0.0,0.0,0.01,15.61,26.33,1091796.0,731603.0,288657.0
2,20220301.0,353.514286,61.028571,0.006329,0.062714,0.0702,0.867114,43174.8,44966.0,42796.9,44428.2,444858241.2,-0.01,0.01,0.01,17.67,29.34,1065354.0,722863.0,280634.0
3,20220228.0,332.333333,61.861111,0.135383,0.033806,0.073306,0.892861,37706.7,44210.4,37446.5,43179.8,564121052.5,0.0,-0.02,-0.01,16.2,24.83,981066.0,702483.0,270918.0
4,20220227.0,128.0,30.0,0.106867,0.022667,0.055,0.922333,39117.3,39864.0,36977.4,37706.7,388368905.7,0.01,0.01,0.0,12.96,21.07,768705.0,583361.0,211164.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604,20200707.0,303.75,79.0,0.22745,0.0,0.06525,0.93475,9342.9,9378.5,9186.7,9254.9,30098389.26,0.01,0.01,0.01,4.7,5.14,969354.0,706177.0,345975.0
605,20200706.0,303.75,79.0,0.22745,0.0,0.06525,0.93475,9072.4,9374.1,9060.0,9342.9,34101061.35,0.01,0.01,0.01,4.88,4.64,989688.0,678157.0,338767.0
606,20200705.0,303.75,79.0,0.22745,0.0,0.06525,0.93475,9135.4,9143.2,8912.0,9072.4,22045158.2,0.01,0.01,0.01,4.32,4.34,771473.0,509206.0,274104.0
607,20200704.0,303.75,79.0,0.22745,0.0,0.06525,0.93475,9059.0,9187.6,9044.3,9135.9,11365928.56,0.01,0.01,0.01,3.81,3.64,799989.0,561108.0,288647.0


### Unique Addresses

In [16]:
KNN_miss_filling = KNNImputer(n_neighbors=2).fit(X_train)
X_train = pd.DataFrame(KNN_miss_filling.transform(X_train), columns=feature_pool)
X_test = pd.DataFrame(KNN_miss_filling.transform(X_test), columns=feature_pool)

In [17]:
print(X_train.isna().sum())
print(X_test.isna().sum())

Time                       0
Favorites                  0
Retweets                   0
Compound                   0
Negative                   0
Positive                   0
Neutral                    0
Open                       0
High                       0
Low                        0
Close                      0
Volume                     0
Funding Rate 0000          0
Funding Rate 0800          0
Funding Rate 1600          0
Bitcoin_SVI                0
Cryptocurrency_SVI         0
No. of Active Addresses    0
No. of Unique Addresses    0
Transactions               0
dtype: int64
Time                       0
Favorites                  0
Retweets                   0
Compound                   0
Negative                   0
Positive                   0
Neutral                    0
Open                       0
High                       0
Low                        0
Close                      0
Volume                     0
Funding Rate 0000          0
Funding Rate 0800          0
F

In [18]:
X_test.to_csv('X_test.csv', index=False)
X_train.to_csv('X_train.csv', index=False)
y_train.to_csv('Y_train.csv', index=False)
y_test.to_csv('Y_test.csv', index=False)

## Preprocessing

In [19]:
# Standardized features:
standardized_features = ['Favorites','Retweets','Compound','Negative','Positive','Neutral','Bitcoin_SVI']     
non_standardized_features = list(set(feature_pool)-set(standardized_features))
    
# Get the scaler based on train set
scaler = preprocessing.StandardScaler().fit(X_train[standardized_features])
print('The mean and standard deviation of training set:', scaler.mean_, scaler.scale_)

# Standardize the features of train and test sets by the scaler of training set
X_train_std=pd.DataFrame(scaler.transform(X_train[standardized_features]))  # transform() return 'numpy.ndarray', not 'DataFrame' or 'Series'
X_train_nstd=pd.DataFrame(X_train[non_standardized_features])

X_test_std=pd.DataFrame(scaler.transform(X_test[standardized_features]))  # return 'numpy.ndarray', not 'DataFrame' or 'Series'
X_test_nstd=pd.DataFrame(X_test[non_standardized_features])

# Rename the columns of X_train_std and X_test_std
X_train_std.columns = X_train_std.columns.map(lambda x: standardized_features[x])
X_test_std.columns = X_test_std.columns.map(lambda x: standardized_features[x])

# Re-combine the normalized and non-normalized data together for both train ans test sets (3 pts)
X_train_std.reset_index(drop=True, inplace=True)
X_train_nstd.reset_index(drop=True, inplace=True)
X_train = pd.concat([X_train_std,X_train_nstd], sort=False,axis=1)
X_test_std.reset_index(drop=True, inplace=True)
X_test_nstd.reset_index(drop=True, inplace=True)
X_test = pd.concat([X_test_std,X_test_nstd], sort=False,axis=1)

# Update the order within feature pool
feature_pool = list(X_train.columns)

The mean and standard deviation of training set: [1.37305098e+03 2.08394184e+02 2.67322500e-01 2.75201429e-02
 1.19668813e-01 8.52826301e-01 1.50408703e+01] [1.70274489e+03 3.03539889e+02 2.26141455e-01 3.98081901e-02
 6.48472402e-02 7.46769667e-02 9.57822930e+00]


In [20]:
X_test.to_csv('../data/X_test_scaled.csv', index=False)
X_train.to_csv('../data/X_train_scaled.csv', index=False)
y_train.to_csv('../data/Y_train_scaled.csv', index=False)
y_test.to_csv('../data/Y_test_scaled.csv', index=False)